Check how many labeled vehicles have a detection mask. Additonal masks are ignored.

Example: 
10 labeled vehicles, 20 masks, 9 labeled vehicles have a corresponding mask
=> 90%

In [1]:
import numpy as np
import plotly.graph_objects as go

import detection
import detection_loader
import providentia_camera_dataset as pcd

In [8]:
scores = []
for i, (detections, label) in enumerate(
    zip(detection_loader.load_detection_lists(), pcd.load_labels())
):
    if not label['labels']:
        scores.append(1)
        continue

    vehicle_detections = [d for d in detections if d.label in [2, 5, 7]]

    mask = detection.merge_masks(vehicle_detections, (1200, 1920))

    corners = [
        [
            l['box3d_projected']['bottom_left_back'],
            l['box3d_projected']['top_right_front'],
        ]
        for l in label['labels']
    ]
    centers = np.array(corners).mean(axis=1) * np.array([2 * 1920, 2 * 1200])
    cx, cy = centers.round().astype(int).T

    cx[cx >= 1920] = 1910
    cy[cy >= 1200] = 1190

    scores.append(mask[cy, cx].mean())


In [9]:
go.Figure(go.Histogram(x=scores, xbins=dict(start=0, end=1.01, size=1 / 100)))